In [1]:
from datasets import load_dataset

dataset = load_dataset("nid989/FNC-1")

c:\Users\ryguy\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (C:/Users/ryguy/.cache/huggingface/datasets/nid989___csv/nid989--FNC-1-ba6c09ed0c9efe30/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 3/3 [00:00<00:00, 10.16it/s]


In [2]:
train = dataset['train']
test = dataset['test']
val = dataset['validation']

In [3]:
train[3]

{'Headline': 'Kimye’s Fault? Anna Wintour’s NYC ‘Vogue’ Offices Hit With Disgusting Rat Infestation: ‘Rat Droppings On Desks’',
 'articleBody': 'YouTube prankster Josh Paler Lin decided give man named Thomas money follow see spends it. His first stop local liquor store instead buying alcohol purchased food gave away others need. A homeless man given chance, touched awful lot surprised people. A viral online video YouTube prankster Josh Paler Lin tells story homeless man named Thomas desire let circumstances keep selfless - using newfound money help himself, others. Josh made video, received 11 million hits Wednesday morning, idea give random homeless person $100 secretly follow see spends it. He meets elderly man gray hair, mustache dark jacket named Thomas holding sign along highway. He tells Josh he\'s trying get enough buy something eat Josh shocks $100. "Oh goodness. Oh way," says Josh pulls five $20 bills wallet. "Oh brother, sure? I\'m starting tear here. That\'s like incredible.

In [4]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import re
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ryguy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Remove puncuation 
# NOTE: HANDLE Sentence Level
import re
import numpy as np
def cleanText(textData):
    # Lowercase
    textDataLower = np.array(list(map(lambda x: x.lower(),textData)))
    # Expand contradictions
    textNoContras = np.array(list(map(lambda x: decontracted(x),textDataLower)))
    # Remove Puncuation
    textDataNoPunc = np.array(list(map(lambda x: re.sub(r'\.|,|\(|\)|\?|!|\n|;|:','',x), textNoContras)))
    # Remove numbers
    textNoNumbers = np.array(list(map(lambda x: re.sub(r'(0|1|2|3|4|5|6|7|8|9|0)+','',x),textDataNoPunc)))
    # Remove extra spaces
    cleanedText = np.array(list(map(lambda x: re.sub(r' +',' ',x),textNoNumbers)))
    # Split
    splitText = np.array(list(map(lambda x: x.split(' '),cleanedText)))
    # Remove stopwords
    english_stopwords = stopwords.words('english')
    return np.array(list(map(lambda x: np.array([t for t in x if t not in english_stopwords]), splitText)))

In [7]:
trainHeadlines = cleanText(train['Headline'])
trainBodies = cleanText(train['articleBody'])
valHeadlines = cleanText(val['Headline'])
valBodies = cleanText(val['articleBody'])
testHeadlines = cleanText(test['Headline'])
testBodies = cleanText(test['articleBody'])
yTrain = train['Stance']
yVal = val['Stance']
yTest = test['Stance']

C:\Users\ryguy\AppData\Local\Temp\ipykernel_10928\1580574302.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  splitText = np.array(list(map(lambda x: x.split(' '),cleanedText)))
C:\Users\ryguy\AppData\Local\Temp\ipykernel_10928\1580574302.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(list(map(lambda x: np.array([t for t in x if t not in english_stopwords]), splitText)))


In [8]:
import gensim.downloader as api
wv = api.load('glove-twitter-50')

In [9]:
vectorSize = 50
def vectorizeWord(word):
    if (word in wv): return wv[word][0:vectorSize]
    return [0]*vectorSize

In [10]:
def vectorizeSent(sent):
    return np.array(list(map(lambda x: vectorizeWord(x),sent)))

In [11]:
vectorTrainHeadlines = np.array(list(map(lambda x: vectorizeSent(x),trainHeadlines)))
vectorTrainBodies = np.array(list(map(lambda x: vectorizeSent(x),trainBodies)))

vectorValHeadlines = np.array(list(map(lambda x: vectorizeSent(x),valHeadlines)))
vectorValBodies = np.array(list(map(lambda x: vectorizeSent(x),valBodies)))

vectorTestHeadlines = np.array(list(map(lambda x: vectorizeSent(x),testHeadlines)))
vectorTestBodies = np.array(list(map(lambda x: vectorizeSent(x),testBodies)))

C:\Users\ryguy\AppData\Local\Temp\ipykernel_10928\3930145267.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vectorTrainHeadlines = np.array(list(map(lambda x: vectorizeSent(x),trainHeadlines)))
C:\Users\ryguy\AppData\Local\Temp\ipykernel_10928\3930145267.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vectorTrainBodies = np.array(list(map(lambda x: vectorizeSent(x),trainBodies)))
C:\Users\ryguy\AppData\Local\Temp\ipykernel_10928\3930145267.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-t

In [12]:
# Find Average Vector of all entries
def getAverageVector(vectors):
    return np.average(vectors,axis=0)

In [13]:
averageVectorTrainHeadlines = np.array(list(map(lambda x: getAverageVector(x),vectorTrainHeadlines)))
averageVectorBodyTrain = np.array(list(map(lambda x: getAverageVector(x),vectorTrainBodies)))

averageVectorValHeadlines = np.array(list(map(lambda x: getAverageVector(x),vectorValHeadlines)))
averageVectorValBodies = np.array(list(map(lambda x: getAverageVector(x),vectorValBodies)))

averageVectorTestHeadlines = np.array(list(map(lambda x: getAverageVector(x),vectorTestHeadlines)))
averageVectorTestBodies = np.array(list(map(lambda x: getAverageVector(x),vectorTestBodies)))

In [14]:
# Headlines = 10
# Body = 90
def pad(arr,size):
    if (len(arr)>size): return arr[0:size]
    elif len(arr)<size: return np.append(arr, np.array((size-len(arr))*[[0]*50]),axis=0)
    return arr

In [15]:
padTrainHeadlines = np.array(list(map(lambda x: pad(x,10),vectorTrainHeadlines)))
padValHeadlines = np.array(list(map(lambda x: pad(x,10),vectorValHeadlines)))
padTestHeadlines = np.array(list(map(lambda x: pad(x,10),vectorTestHeadlines)))

padTrainBodies = np.array(list(map(lambda x: pad(x,90),vectorTrainBodies)))
padValBodies = np.array(list(map(lambda x: pad(x,90),vectorValBodies)))
padTestBodies = np.array(list(map(lambda x: pad(x,90),vectorTestBodies)))

In [16]:
concatTrain = []
for i in range(len(vectorTrainHeadlines)):
    concatTrain.append(np.concatenate((padTrainHeadlines[i],[averageVectorTrainHeadlines[i]+averageVectorBodyTrain[i]],padTrainBodies[i])))
concatTrain = np.array(concatTrain)

In [17]:
concatTrain.shape

(40476, 101, 50)

In [18]:
concatVal = []
for i in range(len(vectorValHeadlines)):
    concatVal.append(np.concatenate((padValHeadlines[i],[averageVectorValHeadlines[i]+averageVectorValBodies[i]],padValBodies[i])))
concatVal = np.array(concatVal)

In [19]:
concatVal.shape

(4498, 101, 50)

In [20]:
concatTest = []
for i in range(len(vectorTestHeadlines)):
    concatTest.append(np.concatenate((padTestHeadlines[i],[averageVectorTestHeadlines[i]+averageVectorTestBodies[i]],padTestBodies[i])))
concatTest = np.array(concatTest)

In [21]:
concatTest.shape

(4998, 101, 50)

In [22]:
def labelsToBinary(labels):
    return np.array(list(map(lambda x: 0 if x==3 else 1,labels)))

In [23]:
yTrainBinary = labelsToBinary(yTrain)
yValBinary = labelsToBinary(yVal)
yTestBinary = labelsToBinary(yTest)

In [24]:
counts = [0,0,0,0]
for i in yTrainBinary:
    counts[i]+=1
counts

[29544, 10932, 0, 0]

In [25]:
from keras.layers import Layer
import keras.backend as K
# https://www.kaggle.com/code/haithemhermessi/attention-mechanism-keras-as-simple-as-possible
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
    def build(self,input_shape):
        self.w=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")
        super(attention,self).build(input_shape)
    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.w)+self.b), axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)
    
    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])
    def get_config(self):
        return super(attention,self).get_config()     

In [41]:
import tensorflow as tf
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(101,50),dtype=tf.float32))

model.add(layers.Bidirectional(layers.LSTM(128,return_sequences=True)))

model.add(layers.Conv1D(64,3))
model.add(layers.MaxPooling1D())

model.add(layers.Bidirectional(layers.LSTM(128,return_sequences=True)))

model.add(layers.Dense(256))
model.add(layers.Dense(256))
model.add(attention())

model.add(layers.Dense(2,activation="softmax"))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 101, 256)          183296    
 al)                                                             
                                                                 
 conv1d (Conv1D)             (None, 99, 64)            49216     
                                                                 
 max_pooling1d (MaxPooling1  (None, 49, 64)            0         
 D)                                                              
                                                                 
 bidirectional_1 (Bidirecti  (None, 49, 256)           197632    
 onal)                                                           
                                                                 
 dense (Dense)               (None, 49, 256)           65792     
                                                        

In [42]:
model.fit(np.array(list(concatTrain)),np.array(list(yTrainBinary)),
          validation_data=(np.array(list(concatVal)),np.array(list(yValBinary))),epochs=15)

Epoch 1/15
1265/1265 [==============================] - 247s 189ms/step - loss: 0.5582 - accuracy: 0.7403 - val_loss: 0.5276 - val_accuracy: 0.7735
Epoch 2/15
1265/1265 [==============================] - 241s 190ms/step - loss: 0.4464 - accuracy: 0.7900 - val_loss: 0.3584 - val_accuracy: 0.8237
Epoch 3/15
1265/1265 [==============================] - 237s 187ms/step - loss: 0.2935 - accuracy: 0.8610 - val_loss: 0.2639 - val_accuracy: 0.8766
Epoch 4/15
1265/1265 [==============================] - 247s 195ms/step - loss: 0.2073 - accuracy: 0.9077 - val_loss: 0.2415 - val_accuracy: 0.8857
Epoch 5/15
1265/1265 [==============================] - 248s 196ms/step - loss: 0.1569 - accuracy: 0.9351 - val_loss: 0.1847 - val_accuracy: 0.9200
Epoch 6/15
1265/1265 [==============================] - 227s 180ms/step - loss: 0.1211 - accuracy: 0.9524 - val_loss: 0.1455 - val_accuracy: 0.9411
Epoch 7/15
1265/1265 [==============================] - 245s 194ms/step - loss: 0.0954 - accuracy: 0.9637 - val_

In [52]:
model.fit(np.array(list(concatTrain)),np.array(list(yTrainBinary)),
          validation_data=(np.array(list(concatVal)),np.array(list(yValBinary))),epochs=3)

Epoch 1/3
1265/1265 [==============================] - 232s 183ms/step - loss: 0.0363 - accuracy: 0.9874 - val_loss: 0.1134 - val_accuracy: 0.9613
Epoch 2/3
1265/1265 [==============================] - 237s 187ms/step - loss: 0.0307 - accuracy: 0.9895 - val_loss: 0.1486 - val_accuracy: 0.9602
Epoch 3/3
1265/1265 [==============================] - 239s 189ms/step - loss: 0.0279 - accuracy: 0.9907 - val_loss: 0.1211 - val_accuracy: 0.9662


In [53]:
preds = model.predict(concatTest)

157/157 [==============================] - 12s 74ms/step


In [54]:
predsActual = np.array(list(map(lambda x: np.argmax(x),preds)))
predsActual

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(yTestBinary,predsActual)

0.974389755902361

In [28]:
# Get data for training the second order model
def getSecondOrderData(x,y):
    newX = []
    newY = []
    for i in range(len(y)):
        if (y[i]!=3):
            newX.append(x[i])
            newY.append(y[i])
    return (np.array(newX),np.array(newY))

In [29]:
xTrainTwo, yTrainTwo = getSecondOrderData(concatTrain,yTrain)
xValTwo, yValTwo = getSecondOrderData(concatVal, yVal)
xTestTwo, yTestTwo = getSecondOrderData(concatTest, yTest)

In [30]:
modelTwo = tf.keras.Sequential()
modelTwo.add(tf.keras.Input(shape=(101,50),dtype=tf.float32))

modelTwo.add(layers.Bidirectional(layers.LSTM(128,return_sequences=True)))

modelTwo.add(layers.Conv1D(64,3))
modelTwo.add(layers.MaxPooling1D())

modelTwo.add(layers.Bidirectional(layers.LSTM(128,return_sequences=True)))

modelTwo.add(layers.Dense(256))
modelTwo.add(layers.Dense(256))
modelTwo.add(attention())

modelTwo.add(layers.Dense(3,activation="softmax"))
modelTwo.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [38]:
modelTwo.fit(np.array(list(xTrainTwo)),np.array(list(yTrainTwo)),
          validation_data=(np.array(list(xValTwo)),np.array(list(yValTwo))),epochs=5)

Epoch 1/5
342/342 [==============================] - 63s 184ms/step - loss: 0.0343 - accuracy: 0.9875 - val_loss: 0.1958 - val_accuracy: 0.9507
Epoch 2/5
342/342 [==============================] - 61s 177ms/step - loss: 0.0356 - accuracy: 0.9879 - val_loss: 0.2157 - val_accuracy: 0.9548
Epoch 3/5
342/342 [==============================] - 60s 177ms/step - loss: 0.0330 - accuracy: 0.9878 - val_loss: 0.2263 - val_accuracy: 0.9431
Epoch 4/5
342/342 [==============================] - 61s 178ms/step - loss: 0.0268 - accuracy: 0.9896 - val_loss: 0.2440 - val_accuracy: 0.9448
Epoch 5/5
342/342 [==============================] - 62s 181ms/step - loss: 0.0373 - accuracy: 0.9862 - val_loss: 0.1630 - val_accuracy: 0.9607


In [66]:
modelTwo.fit(np.array(list(xTrainTwo)),np.array(list(yTrainTwo)),
          validation_data=(np.array(list(xValTwo)),np.array(list(yValTwo))),epochs=2)

Epoch 1/2
342/342 [==============================] - 68s 198ms/step - loss: 0.0303 - accuracy: 0.9895 - val_loss: 0.1889 - val_accuracy: 0.9540
Epoch 2/2
342/342 [==============================] - 67s 197ms/step - loss: 0.0288 - accuracy: 0.9905 - val_loss: 0.1947 - val_accuracy: 0.9582


In [67]:
preds = modelTwo.predict(xTestTwo)
predsActual = np.array(list(map(lambda x: np.argmax(x),preds)))
predsActual

41/41 [==============================] - 3s 77ms/step


array([2, 0, 2, ..., 2, 0, 0], dtype=int64)

In [68]:
from sklearn.metrics import accuracy_score
accuracy_score(yTestTwo,predsActual)

0.968437259430331

In [69]:
# Predictions
# Get First order predictions
finalPreds = []
orderOnePreds = np.array(list(map(lambda x: np.argmax(x),model.predict(concatTest))))
orderTwoPreds = np.array(list(map(lambda x: np.argmax(x),modelTwo.predict(concatTest))))
for i in range(len(orderOnePreds)):
    if (orderOnePreds[i]==0):
        finalPreds.append(3)
    else:
        finalPreds.append(orderTwoPreds[i])

157/157 [==============================] - 10s 66ms/step


In [70]:
counts = [0,0,0,0]
for i in finalPreds:
    counts[i]+=1
counts

[354, 73, 902, 3669]

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(finalPreds,yTest)

0.9667867146858744

In [72]:
import pickle
pickle.dump(model,open('Model1','wb'))
pickle.dump(modelTwo,open('Model2','wb'))